In [1]:
from pacman import *
import numpy as np
import random
import math
from game import Directions
from game import Agent

In [13]:
node_natures = ['Ifle', 'Action', 'Data']
action_subnatures = ['GetFood', 'GetCapsule', 'EscapeFromGhost', 'GetEdibleGhost']
data_subnatures = ['DistToFood', 'DistToCapsule', 'DistToActiveGhost', 'DistToEdibleGhost', 'Constant']

class Cell:
    coordinates = None
    adjacency = np.array([])
    visited = False
    parent = None

class Node:
    nature = None
    subnature = None
    value = None
    parent = None
    children = np.array([])

def get_list_and_dict_of_cells(layout):
    dict_of_cells = dict([])
    list_of_cells = np.array([])

    for x in range(layout.width):
        for y in range(layout.height):
            if not layout.walls[x][y]:#Se não é parede
                new_cell = Cell()
                new_cell.coordinates = (x,y)
                dict_of_cells[str(x) + ',' + str(y)] = new_cell
                list_of_cells = np.append(list_of_cells, new_cell)

    for cell in list_of_cells:
        x = cell.coordinates[0]
        y = cell.coordinates[1]

        if x - 1 >= 0 and not layout.walls[x-1][y]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x-1) + ',' + str(y)])

        if y - 1 >= 0 and not layout.walls[x][y-1]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x) + ',' + str(y-1)])

        if x + 1 <= layout.width and not layout.walls[x+1][y]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x+1) + ',' + str(y)])

        if y + 1 <= layout.height and not layout.walls[x][y+1]:
            cell.adjacency = np.append(cell.adjacency, dict_of_cells[str(x) + ',' + str(y+1)])
    
    return list_of_cells, dict_of_cells
    
#Partial BFS until find final cell
def find_shortest_path(initial_cell_coordinates, final_cell_coordinates, list_of_cells, dict_of_cells):
    
    initial_cell = dict_of_cells[str(initial_cell_coordinates[0]) + ',' + str(initial_cell_coordinates[1])]
    final_cell = dict_of_cells[str(final_cell_coordinates[0]) + ',' + str(final_cell_coordinates[1])]
    
    for cell in list_of_cells:
        cell.visited = False
        cell.parent = None
    
    queue = []
    queue.append(initial_cell)
    
    while not final_cell.visited:
        current_cell = queue.pop(0)
        current_cell.visited = True
        for adjacent_cell in current_cell.adjacency:
            if not adjacent_cell.visited:
                queue.append(adjacent_cell)
                adjacent_cell.parent = current_cell
                
    path = []
    current_cell = final_cell
    while current_cell != initial_cell:
        path.append(current_cell)
        current_cell = current_cell.parent
    path.append(current_cell)
    
    path.reverse()
    
    for i in range(len(path)-1):
        direction = None
        if path[i].coordinates[0] > path[i+1].coordinates[0]:
            direction = Directions.WEST
        elif path[i].coordinates[0] < path[i+1].coordinates[0]:
            direction = Directions.EAST
        elif path[i].coordinates[1] > path[i+1].coordinates[1]:
            direction = Directions.SOUTH
        else:
            direction = Directions.NORTH
        key = str(path[i].coordinates[0]) + ',' + str(path[i].coordinates[1]) + '-' + str(final_cell.coordinates[0]) + ',' + str(final_cell.coordinates[1])
        greedy_moves[key] = direction
    
def build_random_chromosome(root, max_level, max_dist):
    root.nature = 'Ifle'
    c1 = Node()
    c2 = Node()
    c3 = Node()
    c4 = Node()
    
    for c in [c1,c2,c3,c4]:
        c.parent = root

    c1.nature = 'Data'
    c1.subnature = random.choice([item for item in data_subnatures if item != 'Constant'])
      
    c1.nature = 'Data'  
    c2.subnature = random.choice([item for item in data_subnatures if item != c1.subnature])
    if c2.subnature == 'Constant':
        c2.value = random.randint(1, max_dist)
            
    if max_level > 1:
        c3.nature = random.choice(['Action', 'Ifle'])
        c4.nature = random.choice(['Action', 'Ifle'])
    else:
        c3.nature = 'Action'
        c4.nature = 'Action'

    if c3.nature == 'Action':
        c3.subnature = random.choice(action_subnatures)
    elif c3.nature == 'Ifle':
        build_random_chromosome(c3, max_level-1, max_dist)
        
    if c4.nature == 'Action':
        c4.subnature = random.choice([item for item in action_subnatures if item != c3.subnature])
    elif c4.nature == 'Ifle':
        build_random_chromosome(c4, max_level-1, max_dist)
    
    root.children = np.array([c1, c2, c3, c4])

def print_chromosome(root):
    if root.value != None:
        print(root.value)
    elif root.subnature != None:
        print(root.subnature)
    else:
        print(root.nature)
    print()
    for c in root.children:
        print_chromosome(c)
    
class MyAgent(Agent):

    chromosome = None
    
    def find_closest_food(self, state):
        food_list = state.getFood().asList()
        closest_food_coordinates = None
        closest_food_dist = math.inf
        for i in range(len(food_list)):
            current_dist = manhattanDistance(state.getPacmanPosition(), food_list[i])
            if current_dist < closest_food_dist:
                closest_food_dist = current_dist
                closest_food_coordinates = food_list[i]
        return closest_food_coordinates, closest_food_dist
    
    def find_closest_capsule(self, state):
        capsule_list = state.getCapsules()
        closest_capsule_coordinates = None
        closest_capsule_dist = math.inf
        for i in range(len(capsule_list)):
            current_dist = manhattanDistance(state.getPacmanPosition(), capsule_list[i])
            if current_dist < closest_capsule_dist:
                closest_capsule_dist = current_dist
                closest_capsule_coordinates = capsule_list[i]
        return closest_capsule_coordinates, closest_capsule_dist
    
    def find_closest_active_ghost(self, state):
        closest_ghost_coordinates = None
        closest_ghost_dist = math.inf
        for i in range(1, state.getNumAgents()):
            if state.data.agentStates[i].scaredTimer == 0:
                ghost_position = state.getGhostPosition(i)
                ghost_position = (int(ghost_position[0]), int(ghost_position[1]))
                current_dist = manhattanDistance(state.getPacmanPosition(), ghost_position)
                if current_dist < closest_ghost_dist:
                    closest_ghost_dist = current_dist
                    closest_ghost_coordinates = ghost_position
        return closest_ghost_coordinates, closest_ghost_dist
    
    def find_closest_edible_ghost(self, state):
        closest_ghost_coordinates = None
        closest_ghost_dist = math.inf
        for i in range(1, state.getNumAgents()):
            if state.data.agentStates[i].scaredTimer > 1:
                ghost_position = state.getGhostPosition(i)
                ghost_position = (int(ghost_position[0]), int(ghost_position[1]))
                current_dist = manhattanDistance(state.getPacmanPosition(), ghost_position)
                if current_dist < closest_ghost_dist:
                    closest_ghost_dist = current_dist
                    closest_ghost_coordinates = ghost_position
        return closest_ghost_coordinates, closest_ghost_dist    
            
    def parse_chromosome(self, chromosome, state): 
        
        current_node = chromosome
        
        if current_node.nature == 'Ifle':
            children = current_node.children
            for child in children[:2]:
                if child.subnature == 'DistToFood':
                    child.value = self.find_closest_food(state)[1]
                elif child.subnature == 'DistToCapsule':
                    child.value = self.find_closest_capsule(state)[1]
                elif child.subnature == 'DistToActiveGhost':
                    child.value = self.find_closest_active_ghost(state)[1]
                elif child.subnature == 'DistToEdibleGhost':
                    child.value = self.find_closest_edible_ghost(state)[1]
            if children[0].value <= children[1].value:
                return self.parse_chromosome(children[2], state)
            else:
                return self.parse_chromosome(children[3], state)
        elif current_node.nature == 'Action':
            if current_node.subnature == 'GetFood':
                closest_food_coordinates = self.find_closest_food(state)[0]
                current_position = list(state.getPacmanPosition())
                key = str(current_position[0]) + ',' + str(current_position[1]) + '-' + str(closest_food_coordinates[0]) + ',' + str(closest_food_coordinates[1])
                if key not in greedy_moves:
                    find_shortest_path(current_position, closest_food_coordinates, list_of_cells, dict_of_cells)
                action = greedy_moves[key]
                return action
            elif current_node.subnature == 'GetCapsule':
                closest_capsule_coordinates = self.find_closest_capsule(state)[0]
                if closest_capsule_coordinates != None:
                    current_position = list(state.getPacmanPosition())
                    key = str(current_position[0]) + ',' + str(current_position[1]) + '-' + str(closest_capsule_coordinates[0]) + ',' + str(closest_capsule_coordinates[1])
                    if key not in greedy_moves:
                        find_shortest_path(current_position, closest_capsule_coordinates, list_of_cells, dict_of_cells)
                    action = greedy_moves[key]
                else:
                    action = Directions.STOP
                return action 
            elif current_node.subnature == 'GetEdibleGhost':
                closest_edible_ghost_coordinates = self.find_closest_edible_ghost(state)[0]
                if closest_edible_ghost_coordinates != None:
                    current_position = list(state.getPacmanPosition())
                    key = str(current_position[0]) + ',' + str(current_position[1]) + '-' + str(closest_edible_ghost_coordinates[0]) + ',' + str(closest_edible_ghost_coordinates[1])
                    if key not in greedy_moves:
                        find_shortest_path(current_position, closest_edible_ghost_coordinates, list_of_cells, dict_of_cells)
                    action = greedy_moves[key]
                else:
                    action = Directions.STOP
                return action
            elif current_node.subnature == 'EscapeFromGhost':
                current_position = list(state.getPacmanPosition())
                closest_ghost_coordinates = self.find_closest_active_ghost(state)[0]
                if closest_ghost_coordinates != None:
                    max_dist = 0
                    max_dist_move = None
                    for action in state.getLegalPacmanActions():
                        if action == Directions.WEST:
                            next_position = [current_position[0] - 1, current_position[1]]
                        elif action == Directions.EAST:
                            next_position = [current_position[0] + 1, current_position[1]]
                        elif action == Directions.NORTH:
                            next_position = [current_position[0], current_position[1] + 1]
                        elif action == Directions.SOUTH:
                            next_position = [current_position[0], current_position[1] - 1]
                        else:
                            next_position = [current_position[0], current_position[1]]
                        if manhattanDistance(next_position, closest_ghost_coordinates) > max_dist:
                            max_dist_move = action
                            max_dist = manhattanDistance(next_position, closest_ghost_coordinates)
                    action = max_dist_move
                else:
                    action = Directions.STOP
                
                return action

    def getAction(self, state):
        #print ("\nLocation: " + str(state.getPacmanPosition()))
        #print ("Actions available: " + str(state.getLegalPacmanActions()))
        action = self.parse_chromosome(self.chromosome, state)
        return action
        

In [17]:
#seed = 36516285968929
#random.seed(seed)

input_str = "-q -l originalClassic -z 0.75"
args = readCommand(input_str.split())
agent = MyAgent()
args['pacman'] = agent
layout = args['layout']
list_of_cells, dict_of_cells = get_list_and_dict_of_cells(layout)
max_dist = layout.width + layout.height

'''
root = Node()
c1 = Node()
c2 = Node()
c3 = Node()
c4 = Node()
c4_1 = Node()
c4_2 = Node()
c4_3 = Node()
c4_4 = Node()

root.nature = 'Ifle'
c1.nature = 'Data'
c1.subnature = 'DistToActiveGhost'
c2.nature = 'Data'
c2.subnature = 'Constant'
c2.value = 3
c3.nature = 'Action'
c3.subnature = 'EscapeFromGhost'
c4.nature = 'Ifle'

c4_1.nature = 'Data'
c4_1.subnature = 'DistToEdibleGhost'
c4_2.nature = 'Data'
c4_2.subnature = 'Constant'
c4_2.value = 5
c4_3.nature = 'Action'
c4_3.subnature = 'GetEdibleGhost'
c4_4.nature = 'Action'
c4_4.subnature = 'GetFood'

c4.children = np.array([c4_1, c4_2, c4_3, c4_4])

root.children = np.array([c1, c2, c3, c4])

agent.chromosome = root
'''


root = Node()
build_random_chromosome(root, 5, 10)
print_chromosome(root)
agent.chromosome = root

greedy_moves = dict([])

runGames(**args)


Ifle

DistToActiveGhost

DistToCapsule

GetFood

Ifle

DistToFood

DistToActiveGhost

Ifle

DistToActiveGhost

DistToCapsule

GetCapsule

GetFood

Ifle

DistToEdibleGhost

DistToActiveGhost

Ifle

DistToCapsule

6

Ifle

DistToEdibleGhost

6

GetEdibleGhost

GetCapsule

EscapeFromGhost

GetFood

Pacman died! Score: 585
Average Score: 585.0
Scores:        585.0
Win Rate:      0/1 (0.00)
Record:        Loss
